In [1]:
import pandas as pd
import numpy as np
import itertools
import datetime
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
project_id = "perceptive-ivy-290216"

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
query2=f"""
SELECT
A.*, B.LapTime, B.Stint, B.PitOutTime, B.PitInTime,B.Sector1Time, B.Sector2Time, B.Sector3Time, B.Compound, B.TyreLife,B.FreshTyre, B.TrackStatus, B.Position
FROM
`perceptive-ivy-290216.f1_api.race_driver_ahead_2024` A
LEFT JOIN
`perceptive-ivy-290216.f1_api.race_lap_time` B
ON
A.Year=B.Year
AND
A.GP=B.GP
AND
A.Driver=B.Driver
AND
A.LapNumber=B.LapNumber
WHERE
A.GP="Austrian Grand Prix"
# AND
# B.GP="Austrian Grand Prix"
# AND
# A.Driver="SAI"
ORDER BY 1,2,3,4,5,6,7
"""
track2=pd.read_gbq(query2,project_id,dialect='standard')

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_802/368463352.py:24: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  track2=pd.read_gbq(query2,project_id,dialect='standard')


In [4]:
track2[(track2["Driver"]=="RUS")&(track2["LapNumber"]=='70')].head()

,Date,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Time,SessionTime,Distance,DriverAhead,DistanceToDriverAhead,LapNumber,Year,GP,Driver,LapTime,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Compound,TyreLife,FreshTyre,TrackStatus,Position


In [5]:
track2["Time"]=track2['Time'].str.split('days ').str[1]

track2["SessionTime"]=track2['SessionTime'].str.split('days ').str[1]
track2["SessionTime"]=pd.to_datetime(track2['SessionTime'],format='%H:%M:%S.%f')
track2["SessionTime"]=track2["SessionTime"].dt.strftime("%H:%M:%S.%f")

In [6]:
#Convert lapnumber to string for discrete chart coloring
track2["LapNumber"]=track2["LapNumber"].astype(str)
track2["DriverAhead"]=track2["DriverAhead"].astype(str)

In [7]:
track2['Dist_Previous'] = track2.groupby(["LapNumber","Driver"])["Distance"].shift(1)
track2['Dist_Traveled_Incremental'] = track2['Distance']-track2["Dist_Previous"]
track2["Dist_Total_Race"] = track2.groupby(by=["Driver"])["Dist_Traveled_Incremental"].cumsum()
track2['Dist_Diff2'] = track2.groupby(["LapNumber","Driver"])["Distance"].shift(1)
track2['Dist_Diff'] = track2['Distance']-track2["Dist_Diff2"]
track2["Dist_Diff_Sum"] = track2.groupby(by=["LapNumber","Driver"])["Dist_Diff"].cumsum()
  ## Use when not looking at just 1 lap

# #Assign Rank for each entry point
track2["RK"] = track2.groupby(by=["LapNumber","Driver"])["Time"].rank(method="dense", ascending=True)
track2["RK"]= track2["RK"].astype(int)
track2.sort_values(by=["SessionTime","Driver"], inplace=True)
track2['SessionTime_Converted']= pd.to_timedelta(track2["SessionTime"])
track2.loc[:, "SessionTime (s)"] = track2["SessionTime_Converted"].dt.total_seconds()

In [8]:
#Calculate Mean and Median Distance of Driver Ahead for all drivers and laps

mean_distance=pd.DataFrame(track2.groupby(by=["Driver",'LapNumber','Year',"GP"])["DistanceToDriverAhead"].agg(np.nanmean)).reset_index()
mean_distance.columns=["Driver","LapNumber","Year","GP","Distance to Driver Mean"]
median_distance=pd.DataFrame(track2.groupby(by=["Driver",'LapNumber','Year',"GP"])["DistanceToDriverAhead"].agg(np.nanmedian)).reset_index()
median_distance.columns=["Driver","LapNumber","Year","GP","Distance to Driver Median"]
distance_mean_median=mean_distance.merge(median_distance,on=['LapNumber','Driver','Year','GP'])
distance_mean_median["LapNumber"]=distance_mean_median["LapNumber"].astype(int)
distance_mean_median.head()
# points_table=points_table.sort_values(by=["Abbreviation","RoundNumber"])

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_802/1959538915.py:3: FutureWarning: The provided callable <function nanmean at 0x108834860> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  mean_distance=pd.DataFrame(track2.groupby(by=["Driver",'LapNumber','Year',"GP"])["DistanceToDriverAhead"].agg(np.nanmean)).reset_index()
/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_802/1959538915.py:5: FutureWarning: The provided callable <function nanmedian at 0x108834b80> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  median_distance=pd.DataFrame(track2.groupby(by=["Driver",'LapNumber','Year',"GP"])["DistanceToDriverAhead"].agg(np.nanmedian)).reset_index()


,Driver,LapNumber,Year,GP,Distance to Driver Mean,Distance to Driver Median
0,ALB,1,2024,Austrian Grand Prix,7.307855,4.786250
1,ALB,10,2024,Austrian Grand Prix,53.851938,55.409167
2,ALB,11,2024,Austrian Grand Prix,39.100315,41.655000
3,ALB,12,2024,Austrian Grand Prix,95.367784,100.043056
4,ALB,13,2024,Austrian Grand Prix,48.658496,22.482500


In [9]:
#Look at driver ahead plots for single laps
fig_speed=px.line(
    data_frame=track2[(track2["Driver"]=="PIA")&(track2["LapNumber"].isin(['65','66','67']))], 
    x="Dist_Diff_Sum", 
    y="DistanceToDriverAhead",
    color="LapNumber", 
    hover_data=["Stint","GP","Driver","DriverAhead","Compound","Position","nGear","DRS","Throttle","LapTime","LapNumber","PitOutTime","PitInTime","FreshTyre","TyreLife","Sector1Time", "Sector2Time",'Sector3Time'],
    height=900, 
    width=1800,
    template="plotly_dark",
    title="Driver Distance Ahead",
    color_discrete_map={
                  "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
    }
    )
# fig_speed.add_layout_image(
#     dict(
#         source=Image.open("/content/images.jpg"),
#         xref="x", yref="y",
#         x=4500, y=300,
#         sizex=500, sizey=500,
#         xanchor="left", yanchor="middle"
#     )
# )
fig_speed.update_layout(
    xaxis_title="Lap Distance (m)",
    yaxis_title="Distance to Driver Ahead (m)",
)

fig_speed.update_layout(
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="White"
    ),
    title_font_family="PT Sans Narrow"
)
fig_speed.show()

In [10]:
# Distance to driver ahead across multiple laps

fig_all_laps=px.line(
    data_frame=track2[(track2["Driver"]=="HAM")&(track2["LapNumber"].isin(['1','2','3']))], 
    x="Dist_Diff_Sum", 
    y="DistanceToDriverAhead",
    facet_col='LapNumber', 
    facet_col_wrap=1, 
    color='DriverAhead', 
    # symbol='MATCHUP', 
    hover_data=["Stint","GP","Driver","DriverAhead","Compound","Position","nGear","DRS","Throttle","LapTime","LapNumber","PitOutTime","PitInTime","FreshTyre","TyreLife","Sector1Time", "Sector2Time",'Sector3Time'],
    height=1000, width=2000,
    template="plotly_dark",
    title='Distance to Driver Ahead',
    facet_row_spacing=0.014, # default is 0.07 when facet_col_wrap is used
    facet_col_spacing=0.04,
    color_discrete_map={
                 "23": "#64C4FF",
                 "14": "#229971",
                 "77": "#52e252",
                 "10": "#0093cc",
                 "44": "#27F4D2",
                 "27": "#B6BABD",
                 "16": "#E80020",
                 "20": "#B6BABD",
                 "4": "#FF8000",
                 "31": "#0093cc",
                 "11": "#3671C6",
                 "88": "#FF8000",
                 "3": "#6692FF",
                 "63": "#27F4D2",
                 "55": "#E80020",
                 "2": "#37BEDD",
                 "18": "#229971",
                 "22": "#6692FF",
                 "1": "#3671C6",
                 "24": "#52e252"
          }
        )
fig_all_laps.update_layout(
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=20,
        color="White"
    ),
    title_font_family="PT Sans Narrow"
)
fig_all_laps.show()

In [11]:
#Look at driver ahead plots for single laps
fig_all_lap_dist=px.line(
    data_frame=distance_mean_median.sort_values(by=['LapNumber']), 
    x="LapNumber", 
    y="Distance to Driver Mean",
    color="Driver", 
    hover_data=["GP","Year","Driver"],
    height=800, 
    width=1800,
    template="plotly_dark",
    title="Driver Distance Ahead By Lap",
    color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
    }
    )
# fig_speed.add_layout_image(
#     dict(
#         source=Image.open("/content/images.jpg"),
#         xref="x", yref="y",
#         x=4500, y=300,
#         sizex=500, sizey=500,
#         xanchor="left", yanchor="middle"
#     )
# )

fig_all_lap_dist.update_layout(
    xaxis_title="Lap Number",
    yaxis_title="Distance to Driver Ahead (m)",
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="White"
    ),
    title_font_family="PT Sans Narrow"
)

fig_all_lap_dist.update_layout(
    updatemenus=[
        {
            "buttons": [
                {
                    "label": t.name,
                    "method": "restyle",
                    "args": [{"visible": [t2.name == t.name for t2 in fig_all_lap_dist.data]}],
                }
                for t in fig_all_lap_dist.data
            ]
        }
    ]
)

fig_all_lap_dist.show()

In [15]:
fig_driver_ahead_addtl=px.scatter(    
    data_frame=distance_mean_median.sort_values(by=['LapNumber']), 
    x="LapNumber", 
    y="Distance to Driver Median",
    color="Driver", 
    hover_data=["GP","Year","Driver"],
    height=800, 
    width=1800,
    template="plotly_dark",
    title="Driver Distance Ahead By Lap",
    color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
    }
    )
fig_all_lap_dist.add_traces(
    list(fig_driver_ahead_addtl.select_traces())
)

# fig_all_lap_dist.update_layout(
#     xaxis_title="Distance (m)",
#     yaxis_title="Speed (Km/Hr)",
# )
fig_all_lap_dist.update_layout(
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="White"
    ),
    title_font_family="PT Sans Narrow"
)

fig_all_lap_dist.show()

In [24]:
fig_all_lap_dist.update_traces(patch={"line": {"dash": "dash"}}, selector={'name': "Distance to Driver Median"})
